# EE 461P: Data Science Principles
# Assignment 5
## Total points: 70
## Due: Friday, April 24th, submitted via Canvas by 11:59 pm

Your homework should be written in a **Jupyter notebook**. You may work in groups of two if you wish. Your partner needs to be from the same section. Only one student per team needs to submit the assignment on Canvas.  But be sure to include name and UTEID for both students.  Homework groups will be created and managed through Canvas, so please do not arbitrarily change your homework group. If you do change, let the TA know. 

Please ensure that the notebook you have uploaded on Canvas is the correct one, you could download the notebook from Canvas to double check that you have submitted the correct version on your notebook.

Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting. (%matplotlib inline)

In [0]:
%matplotlib inline

### Name(s)
1. 
2. 

The goal of this problem is to explore the effect of feature selection using  the following dataset (same as in HW 4) https://www.kaggle.com/wendykan/lending-club-loan-data

Since the dataset is really huge, we will use only a certain set of features and samples to build our model. We will also use only two classes instead of all.

The modified dataset with reduced number of samples has been prepared for your use. The file can be downloaded using the code below. You can also find it [here](https://drive.google.com/file/d/1Gv_N1rHLqDizxUck6l06BfniR30Pw0Zs/view?usp=sharing) in case you have a different environment and this code does not run.

In [0]:
%%bash
ggID='1Gv_N1rHLqDizxUck6l06BfniR30Pw0Zs'  
ggURL='https://drive.google.com/uc?export=download'  
filename="$(curl -sc /tmp/gcokie "${ggURL}&id=${ggID}" | grep -o '="uc-name.*</span>' | sed 's/.*">//;s/<.a> .*//')"  
getcode="$(awk '/_warning_/ {print $NF}' /tmp/gcokie)"  

if [ -e "${filename}" ]; then
    echo 'File already exists'
else
    curl -Lb /tmp/gcokie "${ggURL}&confirm=${getcode}&id=${ggID}" -o "${filename}"  
fi

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1383      0 --:--:-- --:--:-- --:--:--  1383
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  139M    0  139M    0     0  65.5M      0 --:--:--  0:00:02 --:--:--  175M


In [0]:
import os, sys, re
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

dataset = pd.read_csv('loan.csv')
df = dataset.fillna(0)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (123,124,125,128,129,130,133,139,140,141) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
def LoanResult(status):
    if (status == 'Fully Paid') or (status == 'Current'):
        return 1
    else:
        return 0

df['loan_status'] = df['loan_status'].apply(LoanResult)

In [0]:
df = df[['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade',
             'emp_length', 'home_ownership','annual_inc', 'verification_status', 'loan_status',
             'purpose','addr_state', 'dti','open_acc', 'pub_rec', 'revol_bal', 'revol_util', 
             'initial_list_status', 'recoveries','collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
             'application_type', 'tot_coll_amt', 'tot_cur_bal', 'avg_cur_bal', 'chargeoff_within_12_mths',
             'pub_rec_bankruptcies', 'tax_liens', 'debt_settlement_flag']]

In [0]:
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,open_acc,pub_rec,revol_bal,revol_util,initial_list_status,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,application_type,tot_coll_amt,tot_cur_bal,avg_cur_bal,chargeoff_within_12_mths,pub_rec_bankruptcies,tax_liens,debt_settlement_flag
0,2500,2500,2500.0,36 months,13.56,84.92,C,C1,10+ years,RENT,55000.0,Not Verified,1,debt_consolidation,NY,18.24,9.0,1.0,4341,10.3,w,0.0,0.0,Feb-2019,84.92,Individual,0.0,16901.0,1878.0,0.0,1.0,0.0,N
1,30000,30000,30000.0,60 months,18.94,777.23,D,D2,10+ years,MORTGAGE,90000.0,Source Verified,1,debt_consolidation,LA,26.52,13.0,1.0,12315,24.2,w,0.0,0.0,Feb-2019,777.23,Individual,1208.0,321915.0,24763.0,0.0,1.0,0.0,N
2,5000,5000,5000.0,36 months,17.97,180.69,D,D1,6 years,MORTGAGE,59280.0,Source Verified,1,debt_consolidation,MI,10.51,8.0,0.0,4599,19.1,w,0.0,0.0,Feb-2019,180.69,Individual,0.0,110299.0,18383.0,0.0,0.0,0.0,N
3,4000,4000,4000.0,36 months,18.94,146.51,D,D2,10+ years,MORTGAGE,92000.0,Source Verified,1,debt_consolidation,WA,16.74,10.0,0.0,5468,78.1,w,0.0,0.0,Feb-2019,146.51,Individual,686.0,305049.0,30505.0,0.0,0.0,0.0,N
4,30000,30000,30000.0,60 months,16.14,731.78,C,C4,10+ years,MORTGAGE,57250.0,Not Verified,1,debt_consolidation,MD,26.35,12.0,0.0,829,3.6,w,0.0,0.0,Feb-2019,731.78,Individual,0.0,116007.0,9667.0,0.0,0.0,0.0,N


For simplicity, in this question, we select all columns that do not contain integer of float type of data. Then, one hot encoding is performed.

In [0]:
df_cat = df.select_dtypes(exclude=['int64', 'float64'])
df = pd.get_dummies(df, df_cat.columns.values)

Select the `loan_status` column as the target column.  
## **1. Feature Selection (30 pts)**  
**1.1 (3 pts)** Perform [Min-Max Scaling](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) on the dataset. Continue to use the scaled dataset for Q1 and Q2 of this assignment.   
**1.2 (5 pts)** Split the dataset into train and test set with 25% data in test set and print the total number of features. Use this dataset to create a `RandomForestClassifier(n_estimators=5, random_state=42)` model and print the [classification_report()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) to show the precision, recall and F1 score based on the test set.  
**1.3 (6 pts)** Use $\chi^2$ test to select the top 100, 30 and 10 features using [SelectKBest](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection) module. Train a random forest model and print the [classification_report()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) for each of the trained models.  
**Note:** $\chi^2$ test requires non-negative values only for all features. Since we have performed Min-Max scaling previously (where the default behavior is to scale features between 0 and 1, there should not be any problem performing this test.)   
**1.4 (6 pts)** [Plot the ROC curves](https://github.com/justmarkham/scikit-learn-tips/blob/master/notebooks/21_plot_roc_curve.ipynb) for all of the 4 models.  
**1.5 (8 pts)** Plot the feature importance for each of the 4 random forest models that you have trained to show the top 5 features. ([Get feature names after feature selection](https://stackoverflow.com/a/43765224), [Plot feature importance using Pandas and matplotlib](https://stackoverflow.com/a/51520906))  
**1.6 (2 pts)** What do you observe from the ROC curves?

# Question 2 (20 pts)
Use Principal Component Analysis (PCA) to solve this problem.  
**2.1 (3 pts)** Use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) and reduce the dimension of the dataset to the following components: 100, 30, 10. (3 models)  
**2.2 (6 pts)** Train Random Forest models (each using 5 decision trees (i.e. 5 estimators) for each of the reduced datasets and print the classification report similar to Q1. This will print the class-wise Precision, Recall and F1 score.  
**2.3 (4 pts)** Plot the ROC curves similar to Q1 for all models (trained using dataset containing all dimensions and dataset containing reduced dimensions).  
**2.4 (2 pts)** What do you observe from the ROC curves? Why do you think the classifiers based on PCA are doing worse than those based on feature selection?  
**2.5 (5 pts)** Show the [scree plot](https://www.kindsonthegenius.com/2019/01/12/principal-components-analysispca-in-python-step-by-step/) for the first 10 components of your PCA.

# Question 3 (20 pts)

Use XGBoost to solve this problem.  
**3.1 (2 pts)** Use χ2 test to select the top 30 features using SelectKBest module. (Same as 1.3) \\
**3.2 (10 pts)** Train XGBoost models on the entire dataset and the reduced dataset (with 30 features) \\
(a) Using trees (Model A) \\
(b) Using linear regression (Model B) \\
using XGBClassifier (https://xgboost.readthedocs.io/en/latest/python/python_api.html) and you can specify the weak classifiers to be used using ***booster*** parameter in the XGBClassifier() function. \\
**3.3 (5 pts)** Plot the ROC curves similar to Q1 for all models (trained using dataset containing all dimensions and dataset containing reduced dimensions) for both models A and B.  
**3.4 (3 pts)** Which model performs better (Model A or B) and why?